In [12]:
import pandas as pd 
from pandas.io.json import json_normalize
from ast import literal_eval

In [13]:
MAXROWS = 1e5 # per chunk

In [19]:
train_base = pd.read_csv('data/train_v2.csv', usecols=['fullVisitorId','visitStartTime'], skiprows=0, low_memory=False)
test_base = pd.read_csv('data/test_v2.csv', usecols=['fullVisitorId','visitStartTime'], skiprows=0, low_memory=False)

In [20]:
train_base.head()	

,fullVisitorId,visitStartTime
0,3162355547410993243,1508198450
1,8934116514970143966,1508176307
2,7992466427990357681,1508201613
3,9075655783635761930,1508169851
4,6960673291025684308,1508190552


In [21]:
base_df = pd.concat([train_base, test_base], ignore_index = True, copy = False, join = "outer", sort = True)

In [23]:
df_list = []
for file in ['data/train_v2.csv', 'data/test_v2.csv']:
    reader = pd.read_csv(file, usecols=[6], chunksize = MAXROWS, skiprows=0, low_memory=False)
    for chunk in reader:
        chunk.columns = ['hits']
        chunk['hits'][chunk['hits'] == "[]"] = "[{}]"
        chunk['hits'] = chunk['hits'].apply(literal_eval).str[0]
        chunk = json_normalize(chunk['hits'])

        # Extract the product and promo names from the complex nested structure into a simple flat list:
        if 'product' in chunk.columns:
            #print(chunk['product'][0])
            chunk['v2ProductName'] = chunk['product'].apply(lambda x: [p['v2ProductName'] for p in x] if type(x) == list else [])
            chunk['v2ProductCategory'] = chunk['product'].apply(lambda x: [p['v2ProductCategory'] for p in x] if type(x) == list else [])
            del chunk['product']
        if 'promotion' in chunk.columns:
            #print(chunk['promotion'][0])
            chunk['promoId']  = chunk['promotion'].apply(lambda x: [p['promoId'] for p in x] if type(x) == list else [])
            chunk['promoName']  = chunk['promotion'].apply(lambda x: [p['promoName'] for p in x] if type(x) == list else [])
            del chunk['promotion']

        df_list.append(chunk)
        print(f"Chunk {len(df_list)} completed: {len(chunk)} rows from '{file}'")

Chunk 1 completed: 100000 rows from 'data/train_v2.csv'
Chunk 2 completed: 100000 rows from 'data/train_v2.csv'
Chunk 3 completed: 100000 rows from 'data/train_v2.csv'
Chunk 4 completed: 100000 rows from 'data/train_v2.csv'
Chunk 5 completed: 100000 rows from 'data/train_v2.csv'
Chunk 6 completed: 100000 rows from 'data/train_v2.csv'
Chunk 7 completed: 100000 rows from 'data/train_v2.csv'
Chunk 8 completed: 100000 rows from 'data/train_v2.csv'
Chunk 9 completed: 100000 rows from 'data/train_v2.csv'
Chunk 10 completed: 100000 rows from 'data/train_v2.csv'
Chunk 11 completed: 100000 rows from 'data/train_v2.csv'
Chunk 12 completed: 100000 rows from 'data/train_v2.csv'
Chunk 13 completed: 100000 rows from 'data/train_v2.csv'
Chunk 14 completed: 100000 rows from 'data/train_v2.csv'
Chunk 15 completed: 100000 rows from 'data/train_v2.csv'
Chunk 16 completed: 100000 rows from 'data/train_v2.csv'
Chunk 17 completed: 100000 rows from 'data/train_v2.csv'
Chunk 18 completed: 8337 rows from 'data

In [24]:
df = pd.concat(df_list, ignore_index = True, copy = False, join = "outer", sort = True)

In [25]:
df.shape

(2109926, 79)

In [26]:
df.head()

,appInfo.exitScreenName,appInfo.landingScreenName,appInfo.screenDepth,appInfo.screenName,contentGroup.contentGroup1,contentGroup.contentGroup2,contentGroup.contentGroup3,contentGroup.contentGroup4,contentGroup.contentGroup5,contentGroup.contentGroupUniqueViews1,...,transaction.localTransactionRevenue,transaction.localTransactionShipping,transaction.localTransactionTax,transaction.transactionId,transaction.transactionRevenue,transaction.transactionShipping,transaction.transactionTax,type,v2ProductCategory,v2ProductName
0,shop.googlemerchandisestore.com/google+redesig...,shop.googlemerchandisestore.com/google+redesig...,0,shop.googlemerchandisestore.com/google+redesig...,(not set),Bags,(not set),(not set),(not set),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PAGE,"[Home/Drinkware/Water Bottles and Tumblers/, H...","[Google 17oz Stainless Steel Sport Bottle, 26 ..."
1,shop.googlemerchandisestore.com/asearch.html,shop.googlemerchandisestore.com/home,0,shop.googlemerchandisestore.com/home,(not set),(not set),(not set),(not set),(not set),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PAGE,[],[]
2,shop.googlemerchandisestore.com/google+redesig...,shop.googlemerchandisestore.com/home,0,shop.googlemerchandisestore.com/home,(not set),(not set),(not set),(not set),(not set),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PAGE,[],[]
3,shop.googlemerchandisestore.com/google+redesig...,shop.googlemerchandisestore.com/home,0,shop.googlemerchandisestore.com/home,(not set),(not set),(not set),(not set),(not set),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PAGE,[],[]
4,shop.googlemerchandisestore.com/google+redesig...,shop.googlemerchandisestore.com/home,0,shop.googlemerchandisestore.com/home,(not set),(not set),(not set),(not set),(not set),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PAGE,[],[]


In [27]:
dff = pd.concat([train_base, df], axis=1)

In [28]:
dff.to_csv('data/concat_hit.csv')

In [29]:
dff.shape

(2109926, 81)

In [30]:
dff.head()

,fullVisitorId,visitStartTime,appInfo.exitScreenName,appInfo.landingScreenName,appInfo.screenDepth,appInfo.screenName,contentGroup.contentGroup1,contentGroup.contentGroup2,contentGroup.contentGroup3,contentGroup.contentGroup4,...,transaction.localTransactionRevenue,transaction.localTransactionShipping,transaction.localTransactionTax,transaction.transactionId,transaction.transactionRevenue,transaction.transactionShipping,transaction.transactionTax,type,v2ProductCategory,v2ProductName
0,3162355547410993243,1.508198e+09,shop.googlemerchandisestore.com/google+redesig...,shop.googlemerchandisestore.com/google+redesig...,0,shop.googlemerchandisestore.com/google+redesig...,(not set),Bags,(not set),(not set),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PAGE,"[Home/Drinkware/Water Bottles and Tumblers/, H...","[Google 17oz Stainless Steel Sport Bottle, 26 ..."
1,8934116514970143966,1.508176e+09,shop.googlemerchandisestore.com/asearch.html,shop.googlemerchandisestore.com/home,0,shop.googlemerchandisestore.com/home,(not set),(not set),(not set),(not set),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PAGE,[],[]
2,7992466427990357681,1.508202e+09,shop.googlemerchandisestore.com/google+redesig...,shop.googlemerchandisestore.com/home,0,shop.googlemerchandisestore.com/home,(not set),(not set),(not set),(not set),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PAGE,[],[]
3,9075655783635761930,1.508170e+09,shop.googlemerchandisestore.com/google+redesig...,shop.googlemerchandisestore.com/home,0,shop.googlemerchandisestore.com/home,(not set),(not set),(not set),(not set),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PAGE,[],[]
4,6960673291025684308,1.508191e+09,shop.googlemerchandisestore.com/google+redesig...,shop.googlemerchandisestore.com/home,0,shop.googlemerchandisestore.com/home,(not set),(not set),(not set),(not set),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PAGE,[],[]
